In [ ]:
df = pd.read_csv("/home/workspace/data/B0005_discharge_adjusted.csv", index_col = "datetime_", parse_dates = True)

df.head()

In [ ]:
df_cycle_2["discharge_cycle_id"] = (df_cycle_2["cycle"] // 2).astype(int)  # Maps 2→1, 4→2, 6→3, etc.

In [ ]:
cycle_features = df_cycle_2.groupby("discharge_cycle_id").agg({
    "Voltage_measured": ["mean", "min", "max"],
    "Current_measured": ["mean", "max"],
    "Temperature_measured": ["mean", "max"],
    "Time": ["max"],       # Total discharge time (target)
    "Capacity": ["max"]     # Remaining capacity (target)
}).reset_index()
cycle_features.columns = [
    "discharge_cycle_id",
    "voltage_mean", "voltage_min", "voltage_max",
    "current_mean", "current_max",
    "temperature_mean", "temperature_max",
    "time_total", "capacity"
]

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input

# Create sequences (e.g., use 3 past cycles to predict the next)
sequence_length = 3
X, y = [], []
for i in range(len(cycle_features) - sequence_length):
    X.append(cycle_features.iloc[i:i+sequence_length][[
        "voltage_mean", "current_mean", "temperature_mean", 
        "time_total", "capacity"
    ]].values)
    y.append(cycle_features.iloc[i+sequence_length][["time_total", "capacity"]].values)
X, y = np.array(X), np.array(y)

# Build LSTM model
inputs = Input(shape=(sequence_length, X.shape[2]))
x = LSTM(64, return_sequences=True)(inputs)
x = LSTM(32)(x)
outputs = Dense(2)(x)  # Predict both Time and Capacity
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=100)

In [ ]:
from xgboost import XGBRegressor

# Features: Metrics from past cycles + cycle number
X = cycle_features[["voltage_mean", "current_mean", "discharge_cycle_id"]]
y_time = cycle_features["time_total"]
y_capacity = cycle_features["capacity"]

# Train separate models
model_time = XGBRegressor().fit(X, y_time)
model_capacity = XGBRegressor().fit(X, y_capacity)

In [ ]:
# For LSTM
last_sequence = X[-1].reshape(1, sequence_length, -1)
pred_time, pred_capacity = model.predict(last_sequence)[0]

# For XGBoost
next_cycle_features = np.array([[voltage_mean, current_mean, discharge_cycle_id + 1]])
pred_time = model_time.predict(next_cycle_features)
pred_capacity = model_capacity.predict(next_cycle_features)

In [ ]:
cycle_features["capacity_fade"] = cycle_features["capacity"].iloc[0] - cycle_features["capacity"]

In [ ]:
for i in range(sequence_length, len(cycle_features)):
    train = cycle_features.iloc[:i]
    test = cycle_features.iloc[i]
    # Train and evaluate incrementally